In [1]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classication of newsgroup messages into 20 different categories).
GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)
20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
Currently used dataset is a subset of it.
'''

'This script loads pre-trained word embeddings (GloVe embeddings)\ninto a Keras Embedding layer, and uses it to\ntrain a text classification model on the 20 Newsgroup dataset\n(classication of newsgroup messages into 20 different categories).\nGloVe embedding data can be found at:\nhttp://nlp.stanford.edu/data/glove.6B.zip\n(source page: http://nlp.stanford.edu/projects/glove/)\n20 Newsgroup data can be found at:\nhttp://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html\nCurrently used dataset is a subset of it.\n'

In [2]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Lambda
from keras.layers import Embedding
from keras.models import Model, Sequential
import tensorflow as tf

Using TensorFlow backend.


In [3]:
BASE_DIR = os.getcwd()
GLOVE_DIR = BASE_DIR + '/glove.6B/'
TEXT_DATA_DIR = BASE_DIR + '/20news-bydate-train/'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [7]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')


Indexing word vectors.
Found 400000 word vectors.
Processing text dataset


In [8]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 11314 texts.


In [9]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Found 129096 unique tokens.
Shape of data tensor: (11314, 1000)
Shape of label tensor: (11314, 20)


In [10]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

Preparing embedding matrix.


In [11]:
# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
print('Training model.')

Training model.


In [12]:
model = Sequential()
model.add(Embedding(num_words,
                    EMBEDDING_DIM,
                    input_length=MAX_SEQUENCE_LENGTH,
                    weights=[embedding_matrix],
                    # trainable=False
                    ))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(len(labels_index), activation='softmax'))

In [13]:
from keras import callbacks
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=5, min_lr=0.00001, verbose=1, epsilon=0.001)
early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=1, mode='auto')
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [14]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=200,
          validation_data=(x_val, y_val),
          callbacks=[reduce_lr, early_stop])

Train on 9052 samples, validate on 2262 samples
Epoch 1/200
9052/9052 [==============================] - 12s - loss: 2.9328 - acc: 0.0992 - val_loss: 2.8770 - val_acc: 0.1379
Epoch 2/200
9052/9052 [==============================] - 12s - loss: 2.8055 - acc: 0.1740 - val_loss: 2.7376 - val_acc: 0.2034
Epoch 3/200
9052/9052 [==============================] - 12s - loss: 2.6369 - acc: 0.2654 - val_loss: 2.5544 - val_acc: 0.2891
Epoch 4/200
9052/9052 [==============================] - 13s - loss: 2.4347 - acc: 0.3396 - val_loss: 2.3689 - val_acc: 0.3011
Epoch 5/200
9052/9052 [==============================] - 13s - loss: 2.2366 - acc: 0.3885 - val_loss: 2.1935 - val_acc: 0.4138
Epoch 6/200
9052/9052 [==============================] - 12s - loss: 2.0611 - acc: 0.4401 - val_loss: 2.0476 - val_acc: 0.4054
Epoch 7/200
9052/9052 [==============================] - 12s - loss: 1.9076 - acc: 0.4804 - val_loss: 1.9047 - val_acc: 0.4576
Epoch 8/200
9052/9052 [==============================] - 12s - 